# XYZ Marketing Data Pipeline

See the `/exports` folder for the output of this pipeline.

Start by installing the requirements below. You should do this in a virtualenv.

In [15]:
pip install fastparquet openpyxl pandas

Note: you may need to restart the kernel to use updated packages.


We'll use Pandas to import the data from the sheets we expect. Additional sheets can be 
added below. 

In [16]:
import pandas as pd

filename = "Senior Full Stack- Interview Task Data.xlsx"
transactions = pd.read_excel(filename, sheet_name="transactions")
transactions

,InvoiceNo,StockCode,ProductID,Quantity,UnitPrice,CustomerID
0,1001,ABC123,PRD101,5,10.99,CUST101
1,1002,DEF456,PRD102,3,5.49,CUST102
2,1003,GHI789,PRD103,2,7.99,CUST103
3,1004,ABC123,PRD101,2,10.99,CUST101
4,1005,XYZ789,PRD104,1,15.99,CUST104
5,1006,ABC123,PRD101,4,10.99,CUST101
6,1007,LMN456,PRD105,3,8.49,CUST105
7,1008,DEF456,PRD102,5,5.49,CUST102
8,1009,ABC123,PRD101,3,10.99,CUST101
9,1010,XYZ789,PRD104,2,15.99,CUST104


In [17]:
products = pd.read_excel(filename, sheet_name="products")
products

,ProductID,ProductName
0,PRD101,Baseball Bat
1,PRD102,T-Shirt
2,PRD103,Running Shoes
3,PRD104,Smartphone
4,PRD105,Laptop


In [18]:
customers = pd.read_excel(filename, sheet_name="customers")
customers

,CustomerID,CustomerFirstName,CustomerSurname,CustomerEmail,CustomerCountry
0,CUST101,John,Doe,john.doe@example.com,United Kingdom
1,CUST102,Alice,Smith,alice.smith@example.com,Germany
2,CUST103,Michael,Johnson,michael.johnson@example.com,France
3,CUST104,David,Brown,NaN,United Kingdom
4,CUST105,Robert,Miller,robert.miller@example.com,France
5,CUST106,Matthew,Martin,matthew.martin@example.com,United Kingdom
6,CUST107,Jessica,Thompson,NaN,United Kingdom
7,CUST108,William,Clark,william.clark@example.com,Germany
8,CUST109,Karen,Hall,NaN,France
9,CUST110,Steven,Allen,steven.allen@example.com,United Kingdom


## Manual Task Automation 1:
We want the names of the top five customers by total spend in this data. We'll start by 
creating a new `OrderTotal` column to make the calculation easier. Then we'll group each 
customer together to get their total spend. Finally, we'll sort the customers by their 
total spend and only display the `numberToInclude`, in this case 5.

In [19]:
transactions["OrderTotal"] = transactions["Quantity"] * transactions["UnitPrice"]
orderLedger = transactions[["CustomerID", "OrderTotal"]]
numberToInclude = 5
topCustomerIdsBySpend = (
    orderLedger.groupby("CustomerID")
    .sum()
    .sort_values("OrderTotal", ascending=False)
    .head(numberToInclude)
)
topCustomerIdsBySpend

,OrderTotal
CustomerID,
CUST101,153.86
CUST115,79.95
CUST120,63.96
CUST121,54.95
CUST110,54.95


Since we only have the customer IDs we now use them to look up the customers' first and 
last names.

In [20]:
top_customers = pd.merge(topCustomerIdsBySpend, customers, on="CustomerID")[
    ["CustomerFirstName", "CustomerSurname"]
]
top_customers

,CustomerFirstName,CustomerSurname
0,John,Doe
1,Jennifer,Hall
2,Steven,Hall
3,Donna,Turner
4,Steven,Allen


It's time to export the data to a CSV file. To keep track of the files we will include 
today's date. Ideally the data would include the date it was created, but since it 
doesn't the date this pipeline is run will suffice. We are using `os.sep` for the
directory so that this can run on difference operating systems without issue.

In [21]:
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")

import os

top_customers.to_csv(
    f"exports{os.sep}XYZ-top-customers-{today}.csv", index=False, header=False
)

## Data Transformation 1

We are replacing blank email cells with the text "No Email" so that when the marketing
team run their analysis all of the "No Email" cells will appear together. If instead 
the marketing team wanted to get an accurate count of unique users we could create
unique user IDs for each customer so that they would not have to rely on emails as they
do now.

In [22]:
# Enter value "No Email" in place of NaN in the email column
customers["CustomerEmail"].fillna("No Email", inplace=True)
customers

,CustomerID,CustomerFirstName,CustomerSurname,CustomerEmail,CustomerCountry
0,CUST101,John,Doe,john.doe@example.com,United Kingdom
1,CUST102,Alice,Smith,alice.smith@example.com,Germany
2,CUST103,Michael,Johnson,michael.johnson@example.com,France
3,CUST104,David,Brown,No Email,United Kingdom
4,CUST105,Robert,Miller,robert.miller@example.com,France
5,CUST106,Matthew,Martin,matthew.martin@example.com,United Kingdom
6,CUST107,Jessica,Thompson,No Email,United Kingdom
7,CUST108,William,Clark,william.clark@example.com,Germany
8,CUST109,Karen,Hall,No Email,France
9,CUST110,Steven,Allen,steven.allen@example.com,United Kingdom


## Data transformations 2 and 3
We need to export our data in parquet format.

It also needs to be clear when it was processed. We have three options to do so: We 
could add the date to the file, but would needlessly increase the size if we had to 
include it on each row. We could add something to the file's metadata, which might be 
useful if we knew more about the program the marketing team are using. Third, we can 
include the date once again in the filename. This is the simplest solution and also the 
most widely supported since virtually every program will display filenames and sort 
alphabetically by them. As such, we are using this solution.

First we drop the `OrderTotal` column from transactions, since we don't need it anymore.
The we use Pandas and fastparquet to export the parquet file. We repeat for all the data.

In [23]:
transactions.drop(columns=["OrderTotal"], inplace=True)
transactions

,InvoiceNo,StockCode,ProductID,Quantity,UnitPrice,CustomerID
0,1001,ABC123,PRD101,5,10.99,CUST101
1,1002,DEF456,PRD102,3,5.49,CUST102
2,1003,GHI789,PRD103,2,7.99,CUST103
3,1004,ABC123,PRD101,2,10.99,CUST101
4,1005,XYZ789,PRD104,1,15.99,CUST104
5,1006,ABC123,PRD101,4,10.99,CUST101
6,1007,LMN456,PRD105,3,8.49,CUST105
7,1008,DEF456,PRD102,5,5.49,CUST102
8,1009,ABC123,PRD101,3,10.99,CUST101
9,1010,XYZ789,PRD104,2,15.99,CUST104


In [24]:
transactions_path = f"exports{os.sep}XYZ-transactions-{today}.parquet"
transactions.to_parquet(transactions_path)
pd.read_parquet(transactions_path)

,InvoiceNo,StockCode,ProductID,Quantity,UnitPrice,CustomerID
0,1001,ABC123,PRD101,5,10.99,CUST101
1,1002,DEF456,PRD102,3,5.49,CUST102
2,1003,GHI789,PRD103,2,7.99,CUST103
3,1004,ABC123,PRD101,2,10.99,CUST101
4,1005,XYZ789,PRD104,1,15.99,CUST104
5,1006,ABC123,PRD101,4,10.99,CUST101
6,1007,LMN456,PRD105,3,8.49,CUST105
7,1008,DEF456,PRD102,5,5.49,CUST102
8,1009,ABC123,PRD101,3,10.99,CUST101
9,1010,XYZ789,PRD104,2,15.99,CUST104


In [25]:
customers_path = f"exports{os.sep}XYZ-customers-{today}.parquet"
customers.to_parquet(customers_path)
pd.read_parquet(customers_path)

,CustomerID,CustomerFirstName,CustomerSurname,CustomerEmail,CustomerCountry
0,CUST101,John,Doe,john.doe@example.com,United Kingdom
1,CUST102,Alice,Smith,alice.smith@example.com,Germany
2,CUST103,Michael,Johnson,michael.johnson@example.com,France
3,CUST104,David,Brown,No Email,United Kingdom
4,CUST105,Robert,Miller,robert.miller@example.com,France
5,CUST106,Matthew,Martin,matthew.martin@example.com,United Kingdom
6,CUST107,Jessica,Thompson,No Email,United Kingdom
7,CUST108,William,Clark,william.clark@example.com,Germany
8,CUST109,Karen,Hall,No Email,France
9,CUST110,Steven,Allen,steven.allen@example.com,United Kingdom


In [26]:
products_path = f"exports{os.sep}XYZ-products-{today}.parquet"
products.to_parquet(products_path)
pd.read_parquet(products_path)

,ProductID,ProductName
0,PRD101,Baseball Bat
1,PRD102,T-Shirt
2,PRD103,Running Shoes
3,PRD104,Smartphone
4,PRD105,Laptop


In [27]:
top_customers_parquet_path = f"exports{os.sep}XYZ-top-customers-{today}.parquet"
top_customers.to_parquet(top_customers_parquet_path)
pd.read_parquet(top_customers_parquet_path)

,CustomerFirstName,CustomerSurname
0,John,Doe
1,Jennifer,Hall
2,Steven,Hall
3,Donna,Turner
4,Steven,Allen


## Thanks for reading!
To run this in production use the Python script and Dockerfile as described in the 
readme file.